In [23]:

cosmosContainerName = 'product_details'
cfg = {
  "spark.synapse.linkedService" : "CosmosDb1",
  "spark.cosmos.container" : cosmosContainerName,
  "spark.cosmos.write.strategy": "ItemOverwrite",
  "spark.cosmos.write.bulk.enabled": "true",
}


StatementMeta(pool03, 1, 24, Finished, Available)

In [25]:
df = spark.createDataFrame((("cat-alive", "Schrodinger cat", 2, True), ("cat-dead", "Schrodinger cat", 2, False)))\
    .toDF("id","name","age","isAlive")


StatementMeta(pool03, 1, 26, Finished, Available)

In [26]:
display(df)

StatementMeta(pool03, 1, 27, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8f25c91c-6458-400a-9108-de2ebc33b9d9)

In [27]:
df\
   .write\
   .format("cosmos.oltp")\
   .options(**cfg)\
   .mode("APPEND")\
   .save()

StatementMeta(pool03, 1, 28, Finished, Available)

In [28]:
from pyspark.sql.functions import col

df = spark.read.format("cosmos.oltp").options(**cfg)\
 .option("spark.cosmos.read.inferSchema.enabled", "true")\
 .load()

df.filter(col("isAlive") == True)\
 .show()

StatementMeta(pool03, 1, 29, Finished, Available)

+---------------+-------+----------+-----------+-----+---+-----+---------+---------------+---------+--------+
|           name|isAlive|updated_ts|description|price|age|brand|       id|productCategory|productID|imageURL|
+---------------+-------+----------+-----------+-----+---+-----+---------+---------------+---------+--------+
|Schrodinger cat|   true|      null|       null| null|  2| null|cat-alive|           null|     null|    null|
|        newname|   true|      null|       null| null|  2| null|cat-alive|           null|     null|    null|
+---------------+-------+----------+-----------+-----+---+-----+---------+---------------+---------+--------+

In [29]:
import pyspark.sql.functions as f
df_to_update = df.withColumn('name', f.lit('newname'))

StatementMeta(pool03, 1, 30, Finished, Available)

In [30]:
df_to_update \
        .write \
        .format("cosmos.oltp") \
        .mode("Append") \
        .options(**cfg) \
        .save()

StatementMeta(pool03, 1, 31, Finished, Available)

In [31]:
df = spark.read.format("cosmos.oltp").options(**cfg)\
 .option("spark.cosmos.read.inferSchema.enabled", "true")\
 .load().show()



StatementMeta(pool03, 1, 32, Finished, Available)

+-------+-------+----------------+--------------------+-------+----+---------+--------------------+---------------+---------+--------------------+
|   name|isAlive|      updated_ts|         description|  price| age|    brand|                  id|productCategory|productID|            imageURL|
+-------+-------+----------------+--------------------+-------+----+---------+--------------------+---------------+---------+--------------------+
|newname|   null|1641317730310000|                    |  38.42|null|  Contoso|f16d848b-0b36-426...|           Home|  1005186|     globe black.jpg|
|newname|   null|1641317730310000|  <p>Earphones, c...|  74.99|null|  Contoso|6ad3910e-2cf1-400...|    Electronics|  1002524|mPhone wood table...|
|newname|   null|1641317730310000|                    | 239.42|null|  Contoso|f10eb753-e089-474...| Outdoor Living|  1005161| chairs outdoors.jpg|
|newname|   null|1641317730310000|                    |4719.42|null|  Contoso|3eb74377-2e9a-41c...|           Home|  5